# Условия охватов (Reach conditions)
Пример расчета охватов с дополнительными настройками условий контакта

## Описание задачи и условий расчета

- Регион: Россия 0+
- Период: сентябрь 2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+, Все 18+, Все 18-64, Все 35-55, Все 14-59
- Место просмотра: Все места (Дом+Дача)
- Каналы: Total Channels										
- Статистики: TTV Reach(000), TTV Reach% с условием контакта от 5 до 10 минут непоследовательно

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-09-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter = None

# Указываем список статистик для расчета
statistics = ['TTVReach000', 'TTVReachPer']

# Задаем условия охвата: от 5 до 10 минут непоследовательно
reach_conditions = {
    "filterType": "MINUTES", # Тип условия (минуты/процент от временного интервала): MINUTES или PERCENT
    "viewingType": "NONCONSECUTIVE", # Тип просмотра (последовательный/непоследовательный): CONSECUTIVE или NONCONSECUTIVE 
    # Интервал
    "intervals": [
        {
            "from": 5,
            "to": 10
        }
    ]
}

# Указываем срезы: дата
slices = ['researchDate']

# Задаем условия сортировки: дата (по возр.)
sortings = {'researchDate':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "totalType": "TotalChannels" #база расчета тотал статистик: Total Channels. Возможны опции: TotalTVSet, TotalChannelsThem
}

Сформируем словарь с целевыми аудиториями

In [ ]:
targets = {
    'a. Все 4+' : 'age >= 4',
    'b. Все 18+' : 'age >= 18',
    'c. Все 18-64' : 'age >= 18 AND age <= 64',
    'd. Все 35-55' : 'age >= 35 AND age <= 55',
    'e. Все 14-59' : 'age >= 14 AND age <= 59'
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задание и отправляем на расчет
for target, syntax in targets.items():
    
    # Подставляем значения словаря в параметры 
    project_name = target
    basedemo_filter = syntax
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_timeband_task(task_name=project_name, date_filter=date_filter, 
                                          weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                          company_filter=company_filter, time_filter=time_filter, 
                                          basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                          location_filter=location_filter, slices=slices, 
                                          statistics=statistics, reach_conditions=reach_conditions,
                                          sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 
    
print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

df

## Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
# Формируем сводную таблицу: строки - интервал условия охвата, дни; столбцы - ЦА, значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=['reachInterval','researchDate'], 
                        columns=['prj_name'])
df

In [ ]:
# Поменяем местами уровни в столбцах
df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('08_TTVReach_with_condition_by_date')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)